In [8]:
import fitz  # PyMuPDF

def enumerate_words(pdf_path):
    doc = fitz.open(pdf_path)
    words_with_ids = []
    word_id = 1

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        words = page.get_text("words")  # list of words on the page

        for word in words:
            words_with_ids.append({
                "id": word_id,
                "text": word[4],
                "bbox": word[:4],  # The bounding box of the word
                "page_num": page_num
            })
            word_id += 1

    return words_with_ids

# Extract and enumerate words
pdf_path = "Eren CV Muster.docx.pdf"
words_with_ids = enumerate_words(pdf_path)


In [9]:
def replace_word(pdf_path, output_pdf_path, word_id, replacement_text):
    doc = fitz.open(pdf_path)
    words_with_ids = enumerate_words(pdf_path)

    for word_info in words_with_ids:
        if word_info["id"] == word_id:
            page = doc.load_page(word_info["page_num"])
            bbox = word_info["bbox"]
            rect = fitz.Rect(bbox)
            page.add_redact_annot(rect, text=replacement_text)
            page.apply_redactions()

    doc.save(output_pdf_path)

# Replace a word in the PDF
replace_word("Eren CV Muster.docx.pdf", "updated.pdf", 1, "new text")
